In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Thursday, February 27, 2025
Last Modified: Thursday, August 21, 2025
Program Purpose: To create the map of the biome defintions overlaps with the top 10 absolute and area-weighted contributing ecoregions.
"""

In [ ]:
#Importing the appropriate libraries
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

#Making font times
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#The purpose of this block is to make the different arrays that will represent the different maps of overlaps below

#Making water and other land == 0, just tropical == 1, just dryland == 2, tropical & ecoregion == 3, dryland & ecoregion == 4, and just ecoregion == 5
#The color scheme will go: white, blue, red, green, orange, yellow to be representaive of what the overlap of the colors generates

#Loading in the ecoregions to use with the masking
ecoregions = np.load("wwf_terr_ecos_360_720.npy")
#These are the top 10 ecoregions already calculated
#Only leaving these 10 regions as making all of the rest 0
ecoregions[(ecoregions != 60704.0) & (ecoregions != 80601.0) & (ecoregions != 30704.0) & (ecoregions != 61304.0) & 
                       (ecoregions != 30725.0) & (ecoregions != 30719.0) & (ecoregions != 60173.0) & (ecoregions != 60125.0) & 
                       (ecoregions != 30722.0) & (ecoregions != 80608)] = 0

#Loading in and modifying the MODIS map
modis_data = np.load("MODIS_Map.npy") #The map for the MODIS data
modis_map = np.zeros((360,720))
modis_map[modis_data == 1] = 1 #Just the tropical
modis_map[modis_data == 3] = 2 #Just the dryland
modis_map[ecoregions != 0] = 5 #Just the ecoregion
modis_map[(ecoregions != 0) & (modis_data == 1)] = 3 #Tropical and ecoregion overlap
modis_map[(ecoregions != 0) & (modis_data == 3)] = 4 #Dryland and ecoregion overlap

#Loading in and modifying the Koppen map
file_koppen = nc.Dataset("KG_360x720.nc", "r")
climate_type = file_koppen.variables['kg'][:]
file_koppen.close()
koppen_map = np.zeros((360,720)) #The map for the Koppen data
tropic_climate = climate_type[:2, :, :]
tropic_climate_2d = np.sum(tropic_climate, axis=0) #The tropical regions
dryland_climate = climate_type[2:6, :, :] 
dryland_climate_2d = np.sum(dryland_climate, axis=0) #The dryland regions
koppen_map[tropic_climate_2d>0] = 1 # Tropical is 1
koppen_map[dryland_climate_2d>0] = 2 # Dryland is 2
koppen_map[modis_data == 0] = 0 # Water to account for the coast misalignment
koppen_map[ecoregions != 0] = 5 #Just the ecoregion
koppen_map[(ecoregions != 0) & (tropic_climate_2d>0)] = 3 #Tropical and ecoregion overlap
koppen_map[(ecoregions != 0) & (dryland_climate_2d>0)] = 4 #Dryland and ecoregion overlap

#Loading in and modifying the AI & AEZ map
pangea_array = np.load('aez_tropical_PANGEA.npy')
arid_array = np.load("halfdeg_AI.npy")
ai_aez_map = np.zeros((360,720)) #The map for the AI & AEZ data
ai_aez_map[(0.05 < arid_array) & (arid_array< 0.65)] = 2 #Just the dryland
ai_aez_map[(pangea_array == 5) | (pangea_array == 6) | (pangea_array == 11) | 
           (pangea_array == 12) | (pangea_array == 52)] = 1 #Just the tropical (done after the dryland as the tropical has the priority in this scheme)
ai_aez_map[ecoregions != 0] = 5 #Just the ecoregion
ai_aez_map[(ecoregions != 0) & ((pangea_array == 5) | (pangea_array == 6) | (pangea_array == 11) | 
           (pangea_array == 12) | (pangea_array == 52))] = 3 #Tropical and ecoregion overlap
ai_aez_map[(ecoregions != 0) & ((0.05 < arid_array) & (arid_array< 0.65))] = 4 #Dryland and ecoregion overlap
ai_aez_map[modis_data == 0] = 0 #Mapping out the water

In [ ]:
#The purpose of this block is to create the overlay maps that are reflective of the ecoregion and biome definition maps above

#Putting the 3 maps above into the same list so that it can be looped through to print the global maps of the regions
list_maps = []
list_maps.append(modis_map)
list_maps.append(koppen_map)
list_maps.append(ai_aez_map)
#Doing the 3 loops for the 3 different maps 
i = 0
for map in list_maps:
    # Setting up the map projection
    fig = plt.figure(figsize=(40, 20))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Plotting the global map
    im = ax.imshow(map, cmap=ListedColormap([ 'white', 'blue', 'red', 'green', 'orange', '#FFEA00']),
                extent=[-180, 180, -90, 90], origin='upper', transform=ccrs.PlateCarree(), interpolation='none')
    ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded

    # Adding legend
    Tropical_Bar = mpatches.Patch(facecolor= 'blue', label='Tropical Forest', edgecolor='black')
    Dryland_Bar = mpatches.Patch(facecolor= 'red', label='Dryland', edgecolor='black')
    Tropical_and_Ecoregion_Bar = mpatches.Patch(facecolor= 'green', label='Tropical Forest\n& Ecoregion', edgecolor='black')
    Dryland_and_Ecoregion_Bar = mpatches.Patch(facecolor= 'orange', label='Dryland\n& Ecoregion', edgecolor='black')
    Ecoregion_Bar = mpatches.Patch(facecolor= '#FFEA00', label='Ecoregion', edgecolor='black')

    #Adding the title depending on the map
    if (i == 0):
        plt.title("(b) MODIS", fontsize=70, loc='left', fontweight='bold')
        i+=1
    elif (i==1):
        plt.title("(a) Köppen", fontsize=70, loc='left', fontweight='bold')
        i+=1
    else:
        plt.title("(c) Aridity Index & Agro-Ecological Zones", fontsize=70, loc='left', fontweight='bold')

    #Adding the legend
    plt.legend(handles=[Tropical_Bar, Dryland_Bar, Tropical_and_Ecoregion_Bar, Dryland_and_Ecoregion_Bar, Ecoregion_Bar], loc='lower left', fontsize=45)
    #Showing the plot
    plt.show()

In [ ]:
#This block is concerned with getting the %overlap of the different biome defintion schemes with the top absolute and area-weighted ecoregions

# Area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')

# Loading in the map of the regions.
ecoregions = np.load("wwf_terr_ecos_360_720.npy")

#Loading in the MODIS data to get rid of the water
modis_data = np.load("MODIS_Map.npy")

land_global_area = area_file[modis_data != 0]
global_surface_area = np.sum(land_global_area)

#Making the dict for the top absolute regions
ecoregions_dict = {}
ecoregions_dict["Cerrado"] = 60704
ecoregions_dict["East Siberian"] = 80601
ecoregions_dict["Central Zambezian Miombo"] = 30704
ecoregions_dict["Caatinga"] = 61304
ecoregions_dict["Zambezian Mopane Woodlands"] = 30725
ecoregions_dict["South Miombo"] = 30719
ecoregions_dict["Uatuma-Trombetas"] = 60173
ecoregions_dict["Guianan Moist"] = 60125
ecoregions_dict["West Sudan"] = 30722
ecoregions_dict["Scandinavian & Russian Taiga"] = 80608

#Making the dict for the top area-weighted areas
ecoregions_area_weighted = {}
ecoregions_area_weighted["Western Zambezian grasslands"] = 30724
ecoregions_area_weighted["Eastern Zimbabwe montane forest-grassland mosaic"] = 31006
ecoregions_area_weighted["Zambezian Baikiaea woodlands"] = 30726
ecoregions_area_weighted["Southern Miombo woodlands"] = 30719
ecoregions_area_weighted["Chao Phraya lowland moist deciduous forests"] = 40108
ecoregions_area_weighted["Guianan savanna"] = 60707
ecoregions_area_weighted["Maranhão Babaçu forests"] = 60139
ecoregions_area_weighted["Zambezian coastal flooded savanna"] = 30906
ecoregions_area_weighted["Pernambuco interior forests"] = 60152
ecoregions_area_weighted["Zambezian and Mopane woodlands"] = 30725

#Calculating the Koppen based areas
#Loading in and modifying the Koppen map again to get the correct areas
file_koppen = nc.Dataset("KG_360x720.nc", "r")
climate_type = file_koppen.variables['kg'][:]
file_koppen.close()
koppen_map = np.zeros((360,720)) #The map for the Koppen data
tropic_climate = climate_type[:2, :, :]
tropic_climate_2d = np.sum(tropic_climate, axis=0) #The tropical regions
dryland_climate = climate_type[2:6, :, :] 
dryland_climate_2d = np.sum(dryland_climate, axis=0) #The dryland regions
koppen_map[tropic_climate_2d>0] = 1 # Tropical is 1
koppen_map[dryland_climate_2d>0] = 2 # Dryland is 2

print("Koppen")
#Absolute value areas
for key in ecoregions_dict:
    #Tropical
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_dict[key]) & (koppen_map == 1)] = 1 #Getting the area of the ecoregion in tropical
    area_tropical = temp_map*area_file
    area_tropical = np.sum(area_tropical)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_dict[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the Koppen tropical is: {round((area_tropical / ecoregion_area) *100, 2)}")

    #Dryland
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_dict[key]) & (koppen_map == 2)] = 1 #Getting the area of the ecoregion in dryland
    area_dryland = temp_map*area_file
    area_dryland = np.sum(area_dryland)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_dict[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the Koppen dryland is: {round((area_dryland / ecoregion_area) *100, 2)}\n")

#Area weighted areas
print("\nBREAK...........................\n\n")
for key in ecoregions_area_weighted:
    #Tropical
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_area_weighted[key]) & (koppen_map == 1)] = 1 #Getting the area of the ecoregion in tropical
    area_tropical = temp_map*area_file
    area_tropical = np.sum(area_tropical)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_area_weighted[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the Koppen tropical is: {round((area_tropical / ecoregion_area) *100, 2)}")

    #Dryland
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_area_weighted[key]) & (koppen_map == 2)] = 1 #Getting the area of the ecoregion in dryland
    area_dryland = temp_map*area_file
    area_dryland = np.sum(area_dryland)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_area_weighted[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the Koppen dryland is: {round((area_dryland / ecoregion_area) *100,2)}\n")

#Calculating the MODIS based areas
print("\nMODIS")
#In the MODIS data the tropical region is == 1 and the dryland region is == 3
for key in ecoregions_dict:
    #Tropical
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_dict[key]) & (modis_data == 1)] = 1 #Getting the area of the ecoregion in tropical
    area_tropical = temp_map*area_file
    area_tropical = np.sum(area_tropical)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_dict[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the MODIS tropical is: {round((area_tropical / ecoregion_area) *100,2)}")

    #Dryland
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_dict[key]) & (modis_data == 3)] = 1 #Getting the area of the ecoregion in dryland
    area_dryland = temp_map*area_file
    area_dryland = np.sum(area_dryland)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_dict[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the MODIS dryland is: {round((area_dryland / ecoregion_area) *100,2)}\n")

#Area weighted areas
print("\nBREAK...........................\n\n")
for key in ecoregions_area_weighted:
    #Tropical
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_area_weighted[key]) & (modis_data == 1)] = 1 #Getting the area of the ecoregion in tropical
    area_tropical = temp_map*area_file
    area_tropical = np.sum(area_tropical)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_area_weighted[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the MODIS tropical is: {round((area_tropical / ecoregion_area) *100,2)}")

    #Dryland
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_area_weighted[key]) & (modis_data == 3)] = 1 #Getting the area of the ecoregion in dryland
    area_dryland = temp_map*area_file
    area_dryland = np.sum(area_dryland)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_area_weighted[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the MODIS dryland is: {round((area_dryland / ecoregion_area) *100,2)}\n")

#Calculating the AI&AEZ based areas
print("\nAI&AEZ")
#Setting up the ARID & PANGEA areas now
pangea_array = np.load('aez_tropical_PANGEA.npy')
arid_array = np.load("halfdeg_AI.npy")

# Making the PANGEA array all == 1 where there is actual tropical area
pangea_array[(pangea_array != 5) & (pangea_array != 6)& (pangea_array != 11) & (pangea_array != 12) & (pangea_array != 52)] = 0
pangea_array[pangea_array != 0] = 1

# Making the ARID array all == 2 where there is actual dryland area
arid_array[(0.05 < arid_array) & (arid_array< 0.65)]= -2
arid_array[arid_array != -2] = 0
arid_array[arid_array == -2] = 2
arid_array[300:,:] = 0 # Removing Antarctica.

# Puting the 2 arrays together by giving any overlapping priority to the tropical regions
combined_array = np.zeros((360,720))
combined_array[arid_array == 2] = 2 # Setting arid to 2
combined_array[pangea_array == 1] = 1 # setting tropical to 1
combined_array[modis_data == 0] = 0 #Mapping out the water with 0

for key in ecoregions_dict:
    #Tropical
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_dict[key]) & (combined_array == 1)] = 1 #Getting the area of the ecoregion in tropical
    area_tropical = temp_map*area_file
    area_tropical = np.sum(area_tropical)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_dict[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the AI&AEZ tropical is: {round((area_tropical / ecoregion_area) *100,2)}")

    #Dryland
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_dict[key]) & (combined_array == 2)] = 1 #Getting the area of the ecoregion in dryland
    area_dryland = temp_map*area_file
    area_dryland = np.sum(area_dryland)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_dict[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the AI&AEZ dryland is: {round((area_dryland / ecoregion_area) *100,2)}\n")

#Area weighted areas
print("\nBREAK...........................\n\n")
for key in ecoregions_area_weighted:
    #Tropical
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_area_weighted[key]) & (combined_array == 1)] = 1 #Getting the area of the ecoregion in tropical
    area_tropical = temp_map*area_file
    area_tropical = np.sum(area_tropical)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_area_weighted[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the AI&AEZ tropical is: {round((area_tropical / ecoregion_area) *100,2)}")

    #Dryland
    temp_map = np.zeros((360,720)) #temp map that matches the other maps dimensions
    temp_map[(ecoregions == ecoregions_area_weighted[key]) & (combined_array == 2)] = 1 #Getting the area of the ecoregion in dryland
    area_dryland = temp_map*area_file
    area_dryland = np.sum(area_dryland)
    temp_map = np.zeros((360,720)) #Making another temp map to get the area that the ecoregion takes up
    temp_map[ecoregions == ecoregions_area_weighted[key]] = 1
    ecoregion_area = temp_map*area_file 
    ecoregion_area = np.sum(ecoregion_area)
    print(f"The percentage of the {key} in the AI&AEZ dryland is: {round((area_dryland / ecoregion_area) *100,2)}\n")




In [ ]:
#The purpose of this block is to make the different arrays that will represent the different maps of overlaps for the
# top area-weighted ecoregions below

#Making water and other land == 0, just tropical == 1, just dryland == 2, tropical & ecoregion == 3, dryland & ecoregion == 4, and just ecoregion == 5
#The color scheme will go: white, blue, red, green, orange, yellow to be representaive of what the overlap of the colors generates

#Loading in the ecoregions to use with the masking
ecoregions = np.load("wwf_terr_ecos_360_720.npy")
#These are the top 10 ecoregions already calculated
#Only leaving these 10 regions as making all of the rest 0
ecoregions[(ecoregions != 30724) & (ecoregions != 31006) & (ecoregions != 30726) & (ecoregions != 30719) & 
                       (ecoregions != 40108) & (ecoregions != 60707) & (ecoregions != 60139) & (ecoregions != 30906) & 
                       (ecoregions != 60152) & (ecoregions != 30725)] = 0

#Loading in and modifying the MODIS map
modis_data = np.load("MODIS_Map.npy") #The map for the MODIS data
modis_map = np.zeros((360,720))
modis_map[modis_data == 1] = 1 #Just the tropical
modis_map[modis_data == 3] = 2 #Just the dryland
modis_map[ecoregions != 0] = 5 #Just the ecoregion
modis_map[(ecoregions != 0) & (modis_data == 1)] = 3 #Tropical and ecoregion overlap
modis_map[(ecoregions != 0) & (modis_data == 3)] = 4 #Dryland and ecoregion overlap

#Loading in and modifying the Koppen map
file_koppen = nc.Dataset("KG_360x720.nc", "r")
climate_type = file_koppen.variables['kg'][:]
file_koppen.close()
koppen_map = np.zeros((360,720)) #The map for the Koppen data
tropic_climate = climate_type[:2, :, :]
tropic_climate_2d = np.sum(tropic_climate, axis=0) #The tropical regions
dryland_climate = climate_type[2:6, :, :] 
dryland_climate_2d = np.sum(dryland_climate, axis=0) #The dryland regions
koppen_map[tropic_climate_2d>0] = 1 # Tropical is 1
koppen_map[dryland_climate_2d>0] = 2 # Dryland is 2
koppen_map[modis_data == 0] = 0 # Water to account for the coast misalignment
koppen_map[ecoregions != 0] = 5 #Just the ecoregion
koppen_map[(ecoregions != 0) & (tropic_climate_2d>0)] = 3 #Tropical and ecoregion overlap
koppen_map[(ecoregions != 0) & (dryland_climate_2d>0)] = 4 #Dryland and ecoregion overlap

#Loading in and modifying the AI & AEZ map
pangea_array = np.load('aez_tropical_PANGEA.npy')
arid_array = np.load("halfdeg_AI.npy")
ai_aez_map = np.zeros((360,720)) #The map for the AI & AEZ data
ai_aez_map[(0.05 < arid_array) & (arid_array< 0.65)] = 2 #Just the dryland
ai_aez_map[(pangea_array == 5) | (pangea_array == 6) | (pangea_array == 11) | 
           (pangea_array == 12) | (pangea_array == 52)] = 1 #Just the tropical (done after the dryland as the tropical has the priority in this scheme)
ai_aez_map[ecoregions != 0] = 5 #Just the ecoregion
ai_aez_map[(ecoregions != 0) & ((pangea_array == 5) | (pangea_array == 6) | (pangea_array == 11) | 
           (pangea_array == 12) | (pangea_array == 52))] = 3 #Tropical and ecoregion overlap
ai_aez_map[(ecoregions != 0) & ((0.05 < arid_array) & (arid_array< 0.65))] = 4 #Dryland and ecoregion overlap
ai_aez_map[modis_data == 0] = 0 #Mapping out the water

In [ ]:
#This block is concerned with the creation of the biome overlay map for the top area-weighted ecoregions
#The purpose of this block is to create the overlay maps that are reflective of the ecoregion and biome definition maps above

#Putting the 3 maps above into the same list so that it can be looped through to print the global maps of the regions
list_maps = []
list_maps.append(modis_map)
list_maps.append(koppen_map)
list_maps.append(ai_aez_map)
#Doing the 3 loops for the 3 different maps 
i = 0
for map in list_maps:
    # Setting up the map projection
    fig = plt.figure(figsize=(40, 20))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Plotting the global map
    im = ax.imshow(map, cmap=ListedColormap([ 'white', 'blue', 'red', 'green', 'orange', '#FFEA00']),
                extent=[-180, 180, -90, 90], origin='upper', transform=ccrs.PlateCarree(), interpolation='none')
    ax.set_extent([-90, 110, -30, 25], crs=ccrs.PlateCarree()) #Setting the extent of the map such that the regions of the ecoregions can actually be seen

    # Add legend
    Tropical_Bar = mpatches.Patch(facecolor= 'blue', label='Tropical Forest', edgecolor='black')
    Dryland_Bar = mpatches.Patch(facecolor= 'red', label='Dryland', edgecolor='black')
    Tropical_and_Ecoregion_Bar = mpatches.Patch(facecolor= 'green', label='Tropical Forest\n& Ecoregion', edgecolor='black')
    Dryland_and_Ecoregion_Bar = mpatches.Patch(facecolor= 'orange', label='Dryland\n& Ecoregion', edgecolor='black')
    Ecoregion_Bar = mpatches.Patch(facecolor= '#FFEA00', label='Ecoregion', edgecolor='black')

    #Adding the title depending on the map
    if (i == 0):
        plt.title("(b) MODIS", fontsize=70, loc='left', fontweight='bold')
        i+=1
    elif (i==1):
        plt.title("(a) Köppen", fontsize=70, loc='left', fontweight='bold')
        i+=1
    else:
        plt.title("(c) Aridity Index & Agro-Ecological Zones", fontsize=70, loc='left', fontweight='bold')

    #Adding the legend
    plt.legend(handles=[Tropical_Bar, Dryland_Bar, Tropical_and_Ecoregion_Bar, Dryland_and_Ecoregion_Bar, Ecoregion_Bar], loc='lower left',
               bbox_to_anchor=(0.27,-0.03), fontsize=40)
    #Showing the plot
    plt.show()